## Natural Language SQL on RDS

In [16]:
import os
from sqlalchemy import create_engine, MetaData
from llama_index import SQLDatabase, VectorStoreIndex
from llama_index.indices.struct_store import SQLTableRetrieverQueryEngine, NLSQLTableQueryEngine
from llama_index.objects import SQLTableNodeMapping, ObjectIndex, SQLTableSchema
from langchain import OpenAI

In [17]:
DB_NAME = "your_db_name" # if you don't remember setting one, it is probably postgres
PASSWORD = "your_password"
RDS_ARN = "your_rds_arn"
USER_NAME ="your_username"
PORT = "your_port" # if you don't remember setting one, it is probably 5432

#### Create Engine

In [4]:
pg_uri = f"postgresql+psycopg2://{USER_NAME}:{PASSWORD}@{RDS_ARN}:{PORT}/{DB_NAME}"
engine = create_engine(pg_uri)

#### Create SQLDatabase

In [5]:
# https://docs.sqlalchemy.org/en/20/core/metadata.html#sqlalchemy.schema.MetaData.reflect
# Load all available table definitions from the database.
# MetaData: A collection of Table objects and their associated schema constructs.

metadata_obj = MetaData() 
metadata_obj.reflect(engine)

# Create SQLDatabase
sql_database = SQLDatabase(engine)

In [8]:
## an example of using SQLDatabase to perform sql Query
table_name = "your_table_name"
results = sql_database.run_sql(f"SELECT * FROM {table_name};")
query_result = results[1]["result"]
colnames = results[1]["col_keys"]

#### Natural language SQL

use NLSQLTableQueryEngine to construct natural language queries

In [12]:
query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=[table_name],
)
query_str = "who is the last log in user?"
response = query_engine.query(query_str)

In [ ]:
response.metadata

#### Building Table Index

If we don’t know ahead of time which table we would like to use, and the total size of the table schema overflows your context window size, we wil store the table schema in an index so that during query time we can retrieve the right schema.

In [ ]:
table_node_mapping = SQLTableNodeMapping(sql_database)

table_schema_objs = []

for table in metadata_obj.tables.values():
    table_schema_objs.append(SQLTableSchema(table_name=table.name)) # one SQLTableSchema for each table

# table schema information into a vector index. 
obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)

In [ ]:
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)
response = query_engine.query(query_str)
print(response)

## Natural Language SQL on S3 Data Lake

In [ ]:
from urllib.parse import quote_plus

AWS_REGION = "your_region"
SCHEMA_NAME = "your_Database_name"
S3_STAGING_DIR = "s3://bucket_name/folder_name/"

connect_str = "awsathena+rest://athena.{region_name}.amazonaws.com:443/{schema_name}?s3_staging_dir={s3_staging_dir}"

engine = create_engine(connect_str.format(
        region_name=AWS_REGION,
        schema_name=SCHEMA_NAME,
        s3_staging_dir=quote_plus(S3_STAGING_DIR)
))

metadata_obj = MetaData() 
metadata_obj.reflect(engine) 

sql_database = SQLDatabase(engine)

table_names = [table for table in metadata_obj.tables.values()]

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=table_names,
)
query_str = "who is the last logged in user?"
response = query_engine.query(query_str)

## Customize LLM Used by NLSQL

In [21]:
from langchain_community.chat_models import ChatOpenAI
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(llm=ChatOpenAI(temperature=0, model="gpt-4"))

In [23]:
query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=table_names,
    service_context=service_context
)
query_str = "who is the last logged in user?"
response = query_engine.query(query_str)